In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
import xlsxwriter
from IPython.display import clear_output

In [2]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


In [3]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [4]:
for stock in stocks['Ticker']:
    try:
        data = yf.Ticker(stock).info
        stock_dict = {
            'Ticker': [stock],
            'Stock Price': [data['currentPrice']],
            'Market Capitalization': [data['marketCap']],
            'Number of Shares to Buy': ['N/A']
        }
        df = pd.DataFrame(stock_dict)
        final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)
    except:
        data = yf.Ticker(stock).info
        stock_dict = {
            'Ticker': [stock],
            'Stock Price': [data['currentPrice']],
            'Market Capitalization': [data['marketCap']],
            'Number of Shares to Buy': ['N/A']
        }
        df = pd.DataFrame(stock_dict)
        final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price Market Capitalization Number of Shares to Buy
0        A       127.62           37695885312                     N/A
1      AAL        17.42           11381565440                     N/A
2      AAP        70.73            4204467200                     N/A
3     AAPL       191.94         3018966695936                     N/A
4     ABBV       143.74          253599055872                     N/A
..     ...          ...                   ...                     ...
498    YUM       137.54           38523166720                     N/A
499    ZBH       142.23           29695916032                     N/A
500   ZBRA       306.46           15761329152                     N/A
501   ZION        36.11            5349479936                     N/A
502    ZTS       183.51           84802166784                     N/A

[503 rows x 4 columns]


In [5]:
# portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size = 10000000
portfolio_size

10000000

In [6]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

19880.715705765408

In [7]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,127.62,37695885312,155
1,AAL,17.42,11381565440,1141
2,AAP,70.73,4204467200,281
3,AAPL,191.94,3018966695936,103
4,ABBV,143.74,253599055872,138
...,...,...,...,...
498,YUM,137.54,38523166720,144
499,ZBH,142.23,29695916032,139
500,ZBRA,306.46,15761329152,64
501,ZION,36.11,5349479936,550


In [8]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

In [9]:
bg_color = '#0A0A23'
font_color = '#FFFFFF'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

In [11]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

writer.close()